## A simple model for demand and supply of publicly-provided services in a city

In [10]:
import numpy as np
import pandas as pd
from sklearn import gaussian_process
from matplotlib import pyplot as plt 
from enum import Enum

import xarray as xr


In [48]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: [tuple(np.random.uniform(low=[45.40,9.1],high=[45.50, 9.3], size=2)) for z in range(n)]

In [65]:
get_random_pos(3)

[(45.44299869369619, 9.202496804726204),
 (45.48131353951247, 9.29436103259341),
 (45.48432276428872, 9.264349758775534)]

In [4]:
## Enum classes
class AgeGroup(Enum):
    Newborn= (0, 3)
    Child= (4,15)
    Young= (16,25)
    Junior= (26,35)
    Senior= (36, 50)
    Over50= (50, 64)
    Over65= (66, 80)
    Over80= (81, 200)
    def __init__(self, startAge, endAge):
        self.start = startAge
        self.end = endAge
    
    @staticmethod
    def all():
        return([g for g in AgeGroup])
    @property
    def range(self): return self.end - self.start
    
class SummaryNorm(Enum):
    l1= lambda x: sum(abs(x))
    l2= lambda x: (sum(x**2))**0.5
    lInf= lambda x: max(x)
    

class ServiceType(Enum):
    School = (1, SummaryNorm.l2)
    Football = (2, SummaryNorm.l2)
    SocialSupport = (3, SummaryNorm.l2)
    PoliceStation = (4, SummaryNorm.l2)
    #etc
    def __init__(self, _, aggrNormInput=SummaryNorm.l2):
        self.aggrNorm = aggrNormInput
        # initialise demand factors for each age group
        self.demandFactors = {a: 1 for a in AgeGroup.all()}
    
    def aggregate_units(self, x, axis=1):
        # assumes position are stacked in rows
        return np.apply_along_axis(self.aggrNorm, axis, x)
    
    @staticmethod
    def all():
        return([g for g in ServiceType])
    
#demandFactors = pd.DataFrame(np.ones([len(AgeGroup.all()), len(ServiceType.all())]), 
#                             index=AgeGroup.all(), columns=ServiceType.all())

In [63]:
## ServiceUnit class
class ServiceUnit:
    def __init__(self, service, name='', position=(45.4641, 9.1919), users=AgeGroup.all(), times=[], scale=1):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        assert isinstance(service, ServiceType), 'Service must belong to the Enum'
        self.name = name
        self.service = service
        self.site = position  # a Service can have many sites, and a site is not uniquely assigned to a service
        self.times = times
        
        # how the service availablity area varies for different age groups
        kPropagationTest = 3
        self.propagation ={g: (0.04 + .001*np.round(np.random.normal(),2))*kPropagationTest*scale for g in users} 
        self.kernel = {g: gaussKern(length_scale=l) for g, l in self.propagation.items()}
        
    def evaluate(self, targetPositions, ageGroup):
        assert isinstance(targetPositions[0], tuple),'tuple list format expected for positions'
        reshapedPos = np.array(self.site).reshape(-1,2)
        # evaluate kernel to get level service score. If age group is not served, return 0 as default
        if self.kernel.__contains__(ageGroup):
            score = self.kernel[ageGroup](targetPositions, reshapedPos)
            # check conversion from tuple to nparray
            #targetPositions= np.array(targetPositions)
            #score2 = self.kernel[ageGroup](targetPositions, reshapedPos)
            #assert all(score-score2==0)
        else:
            score = 0
        return np.squeeze(score)
    
    @property
    def users(self): return list(self.propagation.keys())

In [66]:
#outScores = {(a, t): np.zeros(positions.shape[0]) for a in outputAgeGroups for t in outputServices}
#dimSpec = [('position', positions), ('agegroup', outputAgeGroups), ('servicetype', outputServices)]
#outXr = xr.DataArray(np.zeros([positions.shape[0], len(outputAgeGroups), len(outputServices)]),  dimSpec)

### Supply modelling
def evaluate_services_at(positions, unitsList, outputServices= [t for t in ServiceType]):
    # set all age groups as output default
    outputAgeGroups = AgeGroup.all()
    # initialise output
    outScores = {service: pd.DataFrame(np.zeros([len(positions), len(AgeGroup.all())]), 
                                 index=positions, columns=AgeGroup.all()) 
                 for service in outputServices}
    # loop over different services.
    for thisServType in outputServices:
        for thisAgeGroup in outputAgeGroups:
            serviceUnits = [u for u in unitsList if u.service == thisServType]
            if not serviceUnits:
                continue
            unitValues = np.stack(list(map(lambda x: x.evaluate(positions, thisAgeGroup), serviceUnits)), axis=-1)
            # aggregate unit contributions according to the service type norm
            outScores[thisServType][thisAgeGroup] = thisServType.aggregate_units(unitValues)
            
    return outScores
        

In [67]:
test = [ServiceUnit(ServiceType.PoliceStation, 'Duomo'), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', position=(45.43, 9.201))]
evaluate_services_at(get_random_pos(50), test)


{<ServiceType.Football: (2, <function SummaryNorm.<lambda> at 0x7fe3a1694ae8>)>:                                          AgeGroup.Newborn  AgeGroup.Child  \
 (45.44998094716772, 9.241240124490435)                0.0             0.0   
 (45.47221963306177, 9.105677780885566)                0.0             0.0   
 (45.43119637555538, 9.17245011936332)                 0.0             0.0   
 (45.408416657582805, 9.287134721547995)               0.0             0.0   
 (45.46792735021094, 9.210245166576414)                0.0             0.0   
 (45.48397883375984, 9.12935881294412)                 0.0             0.0   
 (45.407850101212304, 9.202011980823006)               0.0             0.0   
 (45.47348892759217, 9.113392846533815)                0.0             0.0   
 (45.47000956414284, 9.172309946042656)                0.0             0.0   
 (45.41704504314629, 9.278918469339818)                0.0             0.0   
 (45.418770837730776, 9.15330965590868)                0.0    

In [68]:
### Demand modelling
class Household:
    def __init__(self, position=get_random_pos(1), members=None):
        if not members: members = {a: 1 for a in AgeGroup.all()}
        self.position = position
        self.members = members
        
def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    # export demand for positions, agegroups and services
    outScores = {service: pd.DataFrame(columns=AgeGroup.all()) 
                 for service in outputServices}
    for household in householdList:
        if out.__contains__(household.position):
            out[household.position] = out.get(household.position, 0)
    

In [73]:
zz = pd.DataFrame(columns=AgeGroup.all())
zz[(1,2),2] = 4
zz

,AgeGroup.Newborn,AgeGroup.Child,AgeGroup.Young,AgeGroup.Junior,AgeGroup.Senior,AgeGroup.Over50,AgeGroup.Over65,AgeGroup.Over80,"((1, 2), 2)"
